___Author(s): Chaojie Feng___<br/>
___Collaborator(s): XXX___<br/>
___External Resources: XXX___<br/>
___Specific Contributions: XXX___<br/>
___Aggregate Contributions: XXX___<br/>


# System Definition
You will consider a 2 wheeled robot similar to the one shown in Fig. 1. It has two wheels of radius r = 20mm,
separated by a distance w = 85mm. It drags a tail for stability.
Figure 1: Two wheeled tank-drive robot
1
Each wheel is powered independently by a bi-directional continuous rotation servo—part number FS90R—with
the angular velocity of the wheel controlled by a signal from the microcontroller. This allows the robot to drive
forwards or backwards at variable speed, or turn with any turning radius. There may be slippage between the wheels
and the floor; assume the resulting effective angular speed of the wheels is normal with a standard deviation of 5%
of the max motor speed.
The robot has two laser range sensors—part number VL53L0X—and an inertial measurement unit (IMU)—part
number MPU-9250. The output of these sensors will be a function of the positional state of the robot within its
environment.
The robot will be driving within a rectangular environment of length L = 750mm and width W = 500mm,
consisting of 4 walls bounding an open space.

In [143]:
import numpy as np
from math import tan,sin,cos,atan
from numpy.random import randn
from filterpy.kalman import ExtendedKalmanFilter as ekf
import collections

class myRobot:
    def __init__(self,wheelRadius = 20, wheelDistance = 85, maxMotorSpeed = np.pi):
        self.wheelRadius = wheelRadius
        self.wheelDistance = wheelDistance
        self.maxMotorSpeed = maxMotorSpeed
        self.errorCoeff = 0.05
        
class myIMU:
    def __init__(self, var = 0.05, bias = 0):
        self.var = var
        self.bias = bias
        
class myLaser:
    def __init__(self, var = 0.05, bias = 0):
        self.var = var
        self.bias = bias

# u is input variable specifies angular valocity of two wheels
def move(x,dt,u,myRobot):
    """
    Input: 
    x: state variable specified by numpy(xpos, ypos, heading)
    u: input control parameter specified as numpy(left_motor_speed, right_motor_speed)
    dt: time increment
    myRobot: the robot specification
    
    Output:
    x_update: updated state variable
    
    NOTE: 
    This is a system model, which means the function doesn't contain any noise. Noise
    will be implemented in input.
    

    """
    if any(u) > myRobot.maxMotorSpeed:
        raise ValueError('motor velocity should not exceed maximum speed')
    
    if x[0] > 750 or x[1] > 500:
        raise ValueError('robot is out of boundary')
    
    r = myRobot.wheelRadius
    d = myRobot.wheelDistance
    

    # wheel's speeds and distances
    lws= u[0]*r
    rws = u[1]*r
    vel = 0.5*(lws+rws)
    lwd = lws*dt
    rwd = rws*dt
    
    # dynamic model (since dynamic model is non-linear equation. We should use EKF)
    dx0 = cos(x[2])*vel*dt
    dx1 = sin(x[2])*vel*dt
    dx2 = atan((rwd-lwd)/d)
    
    
    return x + np.array((dx0,dx1,dx2))

In [144]:
x = np.array((1,1,0.0))
u = np.array((1,2))
IMU = myIMU()
Laser = myLaser()
dt = 0.1
Scott = myRobot()
x_update = move(x,dt,u,Scott)
print(x_update)


[4.         1.         0.02352507]
